In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Importando .csv
df = pd.read_csv('dataset/train_data.csv')

In [31]:
# Implementando baseline
def baseline(R):
    mean = R["rating"].mean(skipna = True)
    users = R.pivot(index='movie_id', columns='user_id', values='rating')
    items = R.pivot(index='user_id', columns='movie_id', values='rating')
    nusers = R.values[:, 0].max()+1
    nitems = R.values[:, 1].max()+1
    bu = np.full(nusers, np.nan, dtype=float)
    bi = np.full(nitems, np.nan, dtype=float)

    for i in items.columns.values:
        bi[i] = np.nanmean(items[i] - mean)

    aux = bi[~np.isnan(bi)]
    for u in users.columns.values:
        bu[u] = np.nanmean(users[u] - mean - aux)

    return { "bu" : bu, "bi": bi, "mean": mean }

In [25]:
# Predizendo nota
def predict(model, u, i):
    return model["mean"] + model["bu"][u] + model["bi"][i]

In [5]:
# Avaliando predições
def rmse(model, test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [32]:
baseline = baseline(df)

In [33]:
rmse(baseline, df.values[401838:])

0.890356975214376